Second try, with a basic model

Imports

In [21]:
import numpy as np
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import torch
from torchvision import models
from torch.utils.data import DataLoader
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

Choose a model

In [22]:
model_name = 'resnet18'
print(f'Using model: {model_name}')

Using model: resnet18


Load and prepare the dataset

In [23]:
image_height = 32
image_width = 150
batch_size = 64
train_size_perc = 0.7

# Define transformations for preprocessing
transform = transforms.Compose([
    transforms.Resize((image_height, image_width)),  # Resize images to a fixed size
    transforms.ToTensor(),                           # Convert images to tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5],      # Normalize pixel values
                         std=[0.5, 0.5, 0.5])])

# Specify the path to your dataset
data_dir = '../fonts'

# Load the dataset using ImageFolder
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Define the sizes of the training, validation, and test sets
train_size = int(train_size_perc * len(dataset))  # 70% of the dataset for training
val_size = int(((1.0 - train_size_perc)/2) * len(dataset))    # 15% for validation
test_size = len(dataset) - train_size - val_size  # Remaining for test (15%)

# Split the dataset into training, validation, and test sets
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [train_size, val_size, test_size])

# Create data loaders for training, validation, and test sets
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Calculate the size of the dataset
num_images = len(dataset)
num_batches = len(train_loader)
print(f'Number of images: {num_images}')
print(f'Number of batches: {num_batches}')
print(f'Number of training images: {len(train_dataset)}')
print(f'Number of validation images: {len(val_dataset)}')
print(f'Number of test images: {len(test_dataset)}')

# Get the class names
class_names = dataset.classes
print(f'Class names: {class_names}')

Number of images: 15000
Number of batches: 165
Number of training images: 10500
Number of validation images: 2250
Number of test images: 2250
Class names: ['Algerian', 'Arial', 'Baskerville', 'Calibry', 'Calligraphy', 'Cambria', 'Comic Sans MS', 'Courier', 'Elephant', 'Fascinate', 'Georgia', 'Helvetica', 'Lucida Bright', 'Nasalization', 'Times New Roman']


Load the model

In [24]:
from sys import platform

# Load the model
model = models.__dict__[model_name](weights=None)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, len(class_names))

device = 'cpu'

print(torch.cuda.is_available())
print(torch.rand(2,3).cuda())

# Move model to the device (CPU or GPU)
if platform != "win32":
    device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
# else:
    # device = torch.device('gpu' if )
model = model.to(device)
print(f'Model moved to {device}')

False


AssertionError: Torch not compiled with CUDA enabled

Train the model

In [15]:
# Define the loss function
criterion = torch.nn.CrossEntropyLoss()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 30
best_val_loss = float('inf')

# Set the model to training mode
model.train()

for epoch in range(num_epochs):
    print(f'Epoch {epoch + 1}\n-------------------------------')
    # Training phase
    model.train()
    train_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        
        # Print training statistics
        if (i + 1) % 10 == 0:
            print(f'Training - Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

    # Calculate average training loss
    train_loss /= len(train_loader.dataset)
    
    # Validation phase
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(val_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)

            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Print validation statistics
    val_loss /= len(val_loader.dataset)
    val_accuracy = correct / total
    print(f'Validation - Epoch [{epoch + 1}/{num_epochs}], Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}')

    # Save the model if validation loss decreases
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), '../models/' + f'{model_name}_font_classifier.pth')
        print(f'Model saved as {model_name}_font_classifier.pth')
        
print('Finished Training')

Epoch 1
-------------------------------
Training - Epoch [1/30], Step [10/165], Loss: 1.7582



KeyboardInterrupt



In [ ]:
# Load the model
model_path = '../models/' + f'{model_name}_font_classifier.pth'
model = models.__dict__[model_name](weights=None)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, len(class_names))
model.load_state_dict(torch.load(model_path))
model = model.to(device)
print(f'Model loaded from {model_path}')

Evaluate the model

In [ ]:
# Set the model to evaluation mode
model.eval()

# Initialize lists for true labels and predicted labels
true_labels = []
predicted_labels = []

# Disable gradient computation for evaluation
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        # Perform inference
        outputs = model(images)
        
        # Compute predicted labels
        _, predicted = torch.max(outputs.data, 1)
        
        # Append true and predicted labels to lists
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

# Convert lists to numpy arrays
true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

# Compute accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print('Accuracy: {:.2f}%'.format(100 * accuracy))

# Compute precision
precision = precision_score(true_labels, predicted_labels, average='weighted')
print('Precision: {:.2f}'.format(precision))

# Compute recall
recall = recall_score(true_labels, predicted_labels, average='weighted')
print('Recall: {:.2f}'.format(recall))

# Compute F1-score
f1 = f1_score(true_labels, predicted_labels, average='weighted')
print('F1-score: {:.2f}'.format(f1))

# Compute confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)
print(f'Confusion Matrix:')
sns.heatmap(conf_matrix, annot=True, cmap='Blues', fmt='g', cbar=False, xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# Compute the ROC curve
# Binarize the true labels
true_labels_binarized = label_binarize(true_labels, classes=range(len(class_names)))

# Binarize the predicted labels
predicted_labels_binarized = label_binarize(predicted_labels, classes=range(len(class_names)))

# Compute the ROC curve
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(len(class_names)):
    fpr[i], tpr[i], _ = roc_curve(true_labels_binarized[:, i], predicted_labels_binarized[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    
# Plot the ROC curve
plt.figure()
for i in range(len(class_names)):
    plt.plot(fpr[i], tpr[i], lw=2, label=f'ROC curve for class {class_names[i]} (area = {roc_auc[i]:0.2f})')
plt.plot([0, 1], [0, 1], 'k--', lw=2)